# Hydrogen isotherms

Relative coverage $\Theta=\frac{S}{S_m}$, with $S$ the surface coverage and $S_m$ the maximum monolayer capacity (parameter of a given surface and material), depends only on the temperature and the surface coverage $S$.

## Dubinin-Radushkevich-Kaganer (DRK) Model

Model describing the physisorption isotherms for low temperatures in the submonolayer region:

$\ln\Theta=-D\left(k_BT\ln\left(\frac{P}{P_0}\right)\right)^2=-B\left(R\,T\ln\left(\frac{P}{P_0}\right)\right)^2$, 

with $k_B$ the Boltzmann constant, $R$ the gas constant, $D$ or $B$ an empirical constant linked to the vaporisation heat of the adsorbate (DR energy)  and $P_0$ the saturated vapor pressure at the temperature $T$.

## Plotting for different fits
### Hobson, 1995
Hobson took $B$ to be $B=3.59\cdot 10^{-6}\,\mathrm{cal^{-2}mol^{2}}$ following Halama and Aggus (https://doi.org/10.1116/1.568583 , 1975). 
### Chill, Wilfert and Bozyk, 2019
Chill, Wilfert and Bozyk measured isotherms between 7 and 18 K on stainless steel and found by fitting $D=3075\,\mathrm{eV}^{-2}$.

In [1]:
from scipy import constants
import matplotlib.pyplot as plt
import numpy as np
import seaborn
import pandas as pd
seaborn.set(style='whitegrid')
seaborn.set_context('paper')
from scipy.optimize import curve_fit
# %matplotlib qt
%matplotlib inline

# Data point from BASE
### Important to bear in mind: BASE never lost a single antiproton to the interaction with residual gas. That means the pressure is only an upper limit and cannot be bounded from below! The error on the coverage on the other hand can be reduced.

## 1. Given
### 1.1 Geometry
* Surfaces:
    * OFHC (Oxygen-free high conductivity copper): $0.080\,\mathrm{m}^2$
    * Sapphire: $0.012\,\mathrm{m}^2$
    * Gold-plated OFHC: $0.009\,\mathrm{m}^2$

In [2]:
a_ofhc=0.08
a_sapphire=0.012
a_au_ofhc=0.009
a_total=a_au_ofhc+a_ofhc+a_sapphire
a_total # m^2

0.10099999999999999

* volume: main pipe with radius $r=34.5\,\mathrm{mm}$ and length $l=231\,\mathrm{mm}$

In [3]:
r=34.5e-3
l=231e-3
v=r**2 * np.pi * l
v # m^3

0.0008637738315210432

### 1.2 Procedure


In [4]:
n_before_pinchoff=0
n_pinchoff=0
time_pinchoffToCooldown=12*3600 # in seconds

## 2. Outgassing
In an unbaked system the outgassing is $H_2O, CO, N_2, ...$ dominant and so overshadow the $H_2$ outgassing. To get the hydrogen outgassing, values [1] measured at baked systems are taken as a first approximate ($TODO$: get better value for unbaked system):

<img src="./outgassing_OFHC.jpeg" width="400">

After bakeout at 250 degree Celcius the total outgassing rate should be equivalent to the hydrogen outgassing rate. 
So it is set to $q_{OFHC}=1.81\cdot 10^{-12}\,\mathrm{Pa\,m^3\,s^{-1}\,m^{-2}}=1.81\cdot 10^{-15}\,\mathrm{mbar\,l\,s^{-1}\,cm^{-2}}$

No values yet found for sapphire and for the effects of gold-plating, so the whole area will be assumed to have the given $q_{OFHC}$.

[1] Y. Koyatsu, H. Miki, F. Watanabe: Measurements of outgassing rate from copper and copper alloy chambers, Vacuum, 47 (6) (1996), pp. 709-711

**New insights:** Copper has higher diffusion rate than e.g. stainless steel at room temperature. Baking has significant impact on the outgassing rate of hydrogen in copper. According to Paolo, the rate is more in the range of: 
$\Rightarrow q_{H_2,\text{unbaked}}\approx 10^{-12}\,\mathrm{mbar\,l\,s^{-1}\,cm^{-2}}$ 

In [5]:
q=1.81e-9 # Pa m^3 s^-1 m^-2
q_molecules=q/(constants.k*296) # at 296 Kelvin
q_molecules

442897859257.8465

## 3. Partial pressure at start of cooldown

Number of hydrogen molecules in system at start of cooldown:

In [6]:
n=n_before_pinchoff+n_pinchoff+q_molecules*a_total*time_pinchoffToCooldown
n

1932451939513835.8

Hydrogen partial pressure at start of cooldown:

In [7]:
p_warm=n*constants.k*296/v
print(p_warm/100.,"mbar")

9.142893326709452e-05 mbar


## 4. Coverage estimate

In [47]:
p_0=4.25e-5 #pascal
d=3075/constants.e**2 # J^-2
s_m=6.45e14*1e4 # m^-2
s_m

6.45e+18

Cold pressure was better than 1E-19 mbar

In [48]:
p_cold=1e-17 #pascal
n_cold=p_cold*v/(constants.k*4.2)
n_cold

148.95924748175594

In [49]:
n_adsorbed=n-n_cold
print("Total number of adsorbed molecules: ", '{:.5E}'.format(n_adsorbed))
s=n_adsorbed/a_total
print("Surface coverage: ", '{:.5E}'.format(s), "molecules per m^-2")
theta=s/s_m
print("Relative surface coverage: ", theta, "<< 1")

Total number of adsorbed molecules:  1.93245E+15
Surface coverage:  1.91332E+16 molecules per m^-2
Relative surface coverage:  0.002966385662005813 << 1


## 5. Surface coverage according to DRK model with fit parameters $D$ and $S_m$

In [50]:
theta_DRK=np.exp(-d*(constants.k*4.2*np.log(p_cold/p_0))**2)
theta_DRK

0.7113589474051248

In [51]:
b_hobson=1./528**2
gas_constant=1.9865
theta_hobson=np.exp(-b_hobson*(gas_constant*4.2*np.log(p_cold/p_0))**2)
theta_hobson

0.8096746277791553

In [52]:
b_hobson*(gas_constant/constants.k)**2

7.425791112504262e+40

In [53]:
d

1.1979103924422136e+41

In [54]:
pressure_plot=np.logspace(-20,-5,40)
theta_4=np.exp(-b_hobson*(gas_constant*4.2*np.log(pressure_plot/p_0))**2)
theta_6=np.exp(-b_hobson*(gas_constant*6*np.log(pressure_plot/0.12369))**2)
theta_10=np.exp(-b_hobson*(gas_constant*10*np.log(pressure_plot/254.03))**2)
plt.loglog(pressure_plot, theta_4, '-', label='4.2 K')
plt.loglog(pressure_plot, theta_6, '-', label='6 K')
plt.loglog(pressure_plot, theta_10, '-', label='10 K')
plt.xlabel('Pressure in Pascal')
plt.ylabel('Relative surface coverage')
plt.title('Hobson theoretical adsorption isotherms\n with fit parameter B taken from Halama and  Aggus (1975) \n https://doi.org/10.1116/1.568583')
plt.legend();

In [55]:
plt.figure(figsize=(8,6))
pressure_plot=np.logspace(-20,-5,40)
theta_4_2=np.exp(-d*(constants.k*4.2*np.log(pressure_plot/p_0))**2)
theta_6_2=np.exp(-d*(constants.k*6*np.log(pressure_plot/0.12369))**2)
theta_10_2=np.exp(-d*(constants.k*10*np.log(pressure_plot/254.03))**2)

plt.loglog(pressure_plot, theta_4, '-',color='royalblue', label='4.2 K, Hobson')
plt.loglog(pressure_plot, theta_4_2, '-', color='lightskyblue', label='4.2 K, Chill, Wilfert and Bozyk')
plt.loglog(pressure_plot, theta_6, '-',color='darkred', label='6 K, Hobson')
plt.loglog(pressure_plot, theta_6_2, '-', color='indianred', label='6 K, Chill, Wilfert and Bozyk')
plt.loglog(pressure_plot, theta_10, '-',color='green', label='10 K, Hobson')
plt.loglog(pressure_plot, theta_10_2, '-',color='darkseagreen', label='10 K, Chill, Wilfert and Bozyk')

plt.loglog(pressure_plot, theta(4.2,pressure_plot*100), color='purple', label='my function')

plt.xlabel('Pressure in Pascal')
plt.ylabel('Relative surface coverage')
plt.title(r'Theoretical hydrogen adsoption isotherms')
plt.legend();

TypeError: 'float' object is not callable

In [ ]:
plt.figure(figsize=(8,6))
pressure_plot=np.logspace(-20,-5,40)
theta_4_2=np.exp(-d*(constants.k*4.2*np.log(pressure_plot/p_0))**2)
theta_6_2=np.exp(-d*(constants.k*6*np.log(pressure_plot/0.12369))**2)
theta_10_2=np.exp(-d*(constants.k*10*np.log(pressure_plot/254.03))**2)

plt.loglog(theta_4, pressure_plot/100., '-',color='royalblue', label='4.2 K, Hobson')
plt.loglog(theta_4_2, pressure_plot/100., '-', color='lightskyblue', label='4.2 K, Chill, Wilfert and Bozyk')
plt.loglog(theta_6,  pressure_plot/100.,'-',color='darkred', label='6 K, Hobson')
plt.loglog(theta_6_2, pressure_plot/100., '-', color='indianred', label='6 K, Chill, Wilfert and Bozyk')
plt.loglog(theta_10,  pressure_plot/100.,'-',color='green', label='10 K, Hobson')
plt.loglog(theta_10_2,  pressure_plot/100.,'-',color='darkseagreen', label='10 K, Chill, Wilfert and Bozyk')

plt.loglog(theta(4.2,pressure_plot*100), pressure(4.2,theta(4.2,pressure_plot*100)) , color='purple', label='my function')


plt.ylabel('Pressure in mbar')
plt.xlabel('Relative surface coverage')
plt.title(r'Theoretical hydrogen adsoption isotherms')
plt.legend();

In [56]:
plt.figure(figsize=(8,6))
plt.loglog(pressure_plot/100., theta_4, '-',color='royalblue', label='4.2 K, Hobson')
plt.loglog(pressure_plot/100., theta_4_2, '-', color='lightskyblue', label='4.2 K, Chill, Wilfert and Bozyk')
plt.xlabel('Pressure in mbar')
plt.ylabel('Relative surface coverage')
plt.title('4.2K only')
plt.grid(axis='y', which='both')
plt.legend();

In [57]:
plt.figure(figsize=(8,6))
plt.loglog(theta_4,pressure_plot/100., '-',color='royalblue', label='4.2 K, Hobson')
plt.loglog(theta_4_2, pressure_plot/100., '-', color='lightskyblue', label='4.2 K, Chill, Wilfert and Bozyk')
plt.ylabel('Pressure in mbar')
plt.xlabel('Relative surface coverage')
plt.title('4.2K only')
plt.grid(axis='x', which='both')
plt.legend();

## 6. Outgassing (or surface area) backwards estimation

In [58]:
theta_chill=np.exp(-d*(constants.k*4.2*np.log(p_cold/p_0))**2)
theta_hobson=np.exp(-b_hobson*(gas_constant*4.2*np.log(p_cold/p_0))**2)
n_adsorbed_chill=theta_chill*s_m*a_total
n_adsorbed_hobson=theta_hobson*s_m*a_total
print("Total number of adsorbed molecules (Hobson): ", '{:.5E}'.format(n_adsorbed_hobson))
print("Total number of adsorbed molecules (Chill et al): ", '{:.5E}'.format(n_adsorbed_chill))

Total number of adsorbed molecules (Hobson):  5.27463E+17
Total number of adsorbed molecules (Chill et al):  4.63415E+17


No adsorption before cooldown, all adsorbed molecules are free in the volume:

In [59]:
p_warm_hobson=n_adsorbed_hobson*constants.k*293.15/v

In [60]:
p_warm_hobson/100

0.024715235737710933

## Fleur's measurments

In [61]:
data=pd.read_csv('./fleurs_measurements.csv')

In [62]:
plt.figure(figsize=(8,6))
plt.loglog(data['Coverage (molecules/cm^2).1']/(3e15), data['Pressure (mbar).1']*100, '.--', label='Fleur\'s data 1')
plt.loglog(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure Barion gauge (mbar)']*100, '.--', label='Fleur\'s data Barion')
plt.loglog(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure RGA (mbar)']*100, '.--', label='Fleur\'s data RGA')
plt.loglog(theta_4,pressure_plot, '-',color='royalblue', label='4.2 K, Hobson')
plt.loglog(theta_4_2, pressure_plot, '-', color='lightskyblue', label='4.2 K, Chill, Wilfert and Bozyk')
plt.ylabel('Pressure in Pascal')
plt.xlabel('Relative surface coverage')
plt.title('4.2K only')
plt.grid(axis='x', which='both')
plt.legend();

In [63]:
plt.figure(figsize=(8,6))
plt.loglog(data['Coverage (molecules/cm^2).1']/(3e15), data['Pressure (mbar).1'], '.--', label='Fleur\'s data 1')
plt.loglog(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure Barion gauge (mbar)'], '.--', label='Fleur\'s data Barion')
plt.loglog(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure RGA (mbar)'], '.--', label='Fleur\'s data RGA')
plt.loglog(theta,p_cold*1e-2, 'D', label='Estimated data point from BASE')
plt.ylabel('Pressure in mbar')
plt.xlabel(r'Relative surface coverage')
plt.title('4.2K')
plt.grid(axis='both', which='minor')
plt.legend();

## Wallen's data

In [64]:
T=4.2 #Kelvin
p_0_mbar=p_0/100
p_0_mbar

4.25e-07

In [65]:
data_wallen=pd.read_csv('./Wallen.csv', delimiter='\t')
offset_mbar=data_wallen['Pressure [mbar]'][23]-p_0_mbar

### Fitting DRK model

In [66]:
def drk(theta, dr_energy,t):
    return p_0_mbar*np.exp(np.sqrt(-np.log(theta))/(-constants.k*T*np.sqrt(dr_energy))+t*1E30)

In [67]:
def lin(x,m,t):
    return m*x+t

In [68]:
thetas=np.logspace(-7,0,100)

In [69]:
x10=np.sqrt(-np.log(data_wallen['Relative surface coverage'][0:10]))
y10=1E30*constants.k*T*np.log(data_wallen['Pressure [mbar]'][0:10]/p_0_mbar)
popt_10,pcov_10=curve_fit(lin,x10,y10)
fit_p10=p_0_mbar*np.exp((popt_10[0]*np.sqrt(-np.log(thetas))+popt_10[1])/(constants.k*T*1E30))
fit_p10_upper=p_0_mbar*np.exp(((popt_10[0]+np.sqrt(pcov_10[0,0]))*np.sqrt(-np.log(thetas))+popt_10[1])/(constants.k*T*1E30))
fit_p10_lower=p_0_mbar*np.exp(((popt_10[0]-np.sqrt(pcov_10[0,0]))*np.sqrt(-np.log(thetas))+popt_10[1])/(constants.k*T*1E30))


In [70]:
x9=np.sqrt(-np.log(data_wallen['Relative surface coverage'][0:9]))
y9=1E30*constants.k*T*np.log(data_wallen['Pressure [mbar]'][0:9]/p_0_mbar)
popt_9,pcov_9=curve_fit(lin,x9,y9)
fit_p9=p_0_mbar*np.exp((popt_9[0]*np.sqrt(-np.log(thetas))+popt_9[1])/(constants.k*T*1E30))

In [71]:
x8=np.sqrt(-np.log(data_wallen['Relative surface coverage'][0:8]))
y8=1E30*constants.k*T*np.log(data_wallen['Pressure [mbar]'][0:8]/p_0_mbar)
popt_8,pcov_8=curve_fit(lin,x8,y8)
fit_p8=p_0_mbar*np.exp((popt_8[0]*np.sqrt(-np.log(thetas))+popt_8[1])/(constants.k*T*1E30))

In [72]:
x7=np.sqrt(-np.log(data_wallen['Relative surface coverage'][0:7]))
y7=1E30*constants.k*T*np.log(data_wallen['Pressure [mbar]'][0:7]/p_0_mbar)
popt_7,pcov_7=curve_fit(lin,x7,y7)
fit_p7=p_0_mbar*np.exp((popt_7[0]*np.sqrt(-np.log(thetas))+popt_7[1])/(constants.k*T*1E30))
fit_p7_upper=p_0_mbar*np.exp(((popt_7[0]+np.sqrt(pcov_7[0,0]))*np.sqrt(-np.log(thetas))+popt_7[1])/(constants.k*T*1E30))
fit_p7_lower=p_0_mbar*np.exp(((popt_7[0]-np.sqrt(pcov_7[0,0]))*np.sqrt(-np.log(thetas))+popt_7[1])/(constants.k*T*1E30))

In [73]:
x4to9=np.sqrt(-np.log(data_wallen['Relative surface coverage'][4:10]))
y4to9=1E30*constants.k*T*np.log(data_wallen['Pressure [mbar]'][4:10]/p_0_mbar)
popt_4to9,pcov_4to9=curve_fit(lin,x4to9,y4to9)
fit_p4to9=p_0_mbar*np.exp((popt_4to9[0]*np.sqrt(-np.log(thetas))+popt_4to9[1])/(constants.k*T*1E30))

In [74]:
x4to8=np.sqrt(-np.log(data_wallen['Relative surface coverage'][4:9]))
y4to8=1E30*constants.k*T*np.log(data_wallen['Pressure [mbar]'][4:9]/p_0_mbar)
popt_4to8,pcov_4to8=curve_fit(lin,x4to8,y4to8)
fit_p4to8=p_0_mbar*np.exp((popt_4to8[0]*np.sqrt(-np.log(thetas))+popt_4to8[1])/(constants.k*T*1E30))

In [75]:
x4to7=np.sqrt(-np.log(data_wallen['Relative surface coverage'][4:8]))
y4to7=1E30*constants.k*T*np.log(data_wallen['Pressure [mbar]'][4:8]/p_0_mbar)
popt_4to7,pcov_4to7=curve_fit(lin,x4to7,y4to7)
fit_p4to7=p_0_mbar*np.exp((popt_4to7[0]*np.sqrt(-np.log(thetas))+popt_4to7[1])/(constants.k*T*1E30))
fit_p4to7_upper=p_0_mbar*np.exp(((popt_4to7[0]+np.sqrt(pcov_4to7[0,0]))*np.sqrt(-np.log(thetas))+popt_4to7[1])/(constants.k*T*1E30))
fit_p4to7_lower=p_0_mbar*np.exp(((popt_4to7[0]-np.sqrt(pcov_4to7[0,0]))*np.sqrt(-np.log(thetas))+popt_4to7[1])/(constants.k*T*1E30))                                

In [81]:
plt.figure(figsize=(8,6))

plt.loglog(thetas,fit_p10,'--', label='Fit with 10 data points')
plt.loglog(thetas,fit_p9,'--', label='Fit with 9 data points')
# plt.loglog(thetas,fit_p8,'--', label='Fit with 8 data points')
plt.loglog(thetas,fit_p7,'--', label='Fit with 7 data points')
plt.loglog(thetas,fit_p4to9,'--', label='Fit with data points 4 to 9')
# plt.loglog(thetas,fit_p4to8,'--', label='Fit with data points 4 to 8')
plt.loglog(thetas,fit_p4to7,'--', label='Fit with data points 4 to 7')


plt.plot(data_wallen['Relative surface coverage'][4:],data_wallen['Pressure [mbar]'][4:], 'r.-', label='Wallen\'s data on Cu plated stainless steel')
plt.loglog(data_wallen['Relative surface coverage'][:4],data_wallen['Pressure [mbar]'][:4], 'r*', label='Wallen\'s data less than 10% over background pressure')

plt.loglog(theta,p_cold*1e-2, 'D',color='purple', label='Estimated data point from BASE')

plt.ylabel('Pressure in mbar')
plt.xlabel(r'Relative surface coverage')
plt.title('Hydrogen on copper at 4.2K')
plt.grid(axis='both', which='minor')
plt.legend();

In [77]:
plt.figure(figsize=(8,6))
thetas=np.logspace(-7,0,100)

plt.loglog(thetas,fit_p10,'--', label='Fit with 10 data points')
# plt.loglog(thetas,fit_p10_upper,'-', label='Fit with 10 data points upper')
# plt.loglog(thetas,fit_p10_lower,'-', label='Fit with 10 data points lower')

plt.loglog(thetas,fit_p9,'--', label='Fit with 9 data points')
plt.loglog(thetas,fit_p8,'--', label='Fit with 8 data points')
plt.loglog(thetas,fit_p7,'--', label='Fit with 7 data points')
# plt.loglog(thetas,fit_p7_upper,'--', label='Fit with 7 data points upper')
# plt.loglog(thetas,fit_p7_lower,'--', label='Fit with 7 data points lower')
plt.loglog(thetas,fit_p4to9,'-', label='Fit with data points 4 to 9')
plt.loglog(thetas,fit_p4to8,'-', label='Fit with data points 4 to 8')
plt.loglog(thetas,fit_p4to7,'-', label='Fit with data points 4 to 7')
# plt.loglog(thetas,fit_p4to7_upper,'-.', label='Fit with data points 4 to 7 upper')
# plt.loglog(thetas,fit_p4to7_lower,'-.', label='Fit with data points 4 to 7 lower')


plt.plot(data_wallen['Relative surface coverage'][4:],data_wallen['Pressure [mbar]'][4:], 'r.-', label='Wallen\'s data on Cu plated stainless steel')
plt.loglog(data_wallen['Relative surface coverage'][:4],data_wallen['Pressure [mbar]'][:4], 'r*', label='Wallen\'s data less than 10% over background pressure')

plt.loglog(theta,p_cold*1e-2, 'D',color='purple', label='Estimated data point from BASE')

plt.ylabel('Pressure in mbar')
plt.xlabel(r'Relative surface coverage')
plt.title('Hydrogen on copper at 4.2K')
plt.grid(axis='both', which='minor')
plt.legend();

In [78]:
np.sqrt(pcov_10[0,0])

13623155.966969173

## Comparison of all data and fits

In [79]:
# %matplotlib qt
plt.figure(figsize=(8,6))
plt.rcParams.update(plt.rcParamsDefault)

pressure_plot2=np.logspace(-21,-9,40)
theta_hobson=np.exp(-b_hobson*(gas_constant*4.2*np.log(pressure_plot2/(p_0*1e-2)))**2)
theta_wilfert=np.exp(-d*(constants.k*4.2*np.log(pressure_plot2/(p_0*1e-2)))**2)

# plt.loglog(data['Coverage (molecules/cm^2).1']/(3e15), data['Pressure (mbar).1'], '.--' )
#plt.errorbar(data['Coverage (molecules/cm^2).1']/(3e15), data['Pressure (mbar).1'],xerr=data['Coverage (molecules/cm^2).1']/(3e15)*0.20, yerr= data['Pressure (mbar).1']*0.99,fmt='^-',color='brown', alpha=0.5)
plt.loglog(data['Coverage (molecules/cm^2).1']/(3e15), data['Pressure (mbar).1'],'^-',color='brown',label='Fleur\'s data 1')
#plt.errorbar(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure Barion gauge (mbar)'],xerr=data['Coverage (molecules/cm^2).2']/(3e15)*0.20, yerr= data['Pressure Barion gauge (mbar)']*0.99,fmt='o-',color='orange',alpha=0.5)
plt.loglog(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure Barion gauge (mbar)'],'o-',color='orange' ,label='Fleur\'s data Barion gauge' )
# plt.loglog(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure RGA (mbar)'], '.--', label='Fleur\'s data RGA')
#plt.errorbar(data_wallen['Relative surface coverage'][4:],data_wallen['Pressure [mbar]'][4:],xerr=data_wallen['Relative surface coverage'][4:]*0.20,yerr=data_wallen['Pressure [mbar]'][4:]*0.99, fmt='g-',alpha=0.5)
plt.loglog(data_wallen['Relative surface coverage'][4:],data_wallen['Pressure [mbar]'][4:], 'gD-', label='Wallen\'s data on Cu plated stainless steel')
plt.loglog(data_wallen['Relative surface coverage'][:4],data_wallen['Pressure [mbar]'][:4], 'g*', label='Wallen\'s data less than 10% over background pressure')

plt.loglog(theta,p_cold*1e-2, 'D',color='purple', label='Estimated data point from BASE')
plt.fill_between(x=[4,theta],y1=[p_cold*1e-2,p_cold*1e-2],y2=[1e-21,1e-21],color='purple',alpha=0.5)

plt.loglog(theta_hobson,pressure_plot2, '--',color='royalblue', label='Prediction Hobson')
plt.loglog(theta_wilfert, pressure_plot2, '-.', color='lightskyblue', label='Prediction from fit of data from Chill, Wilfert and Bozyk')

plt.ylim(1e-21)
plt.xlim(2e-3,4)
plt.ylabel('Pressure in mbar')
plt.xlabel(r'Relative surface coverage')
plt.title('Hydrogen on copper at 4.2K')
plt.grid(axis='both', which='both')
plt.tight_layout()
plt.legend();

In [80]:
%matplotlib qt
seaborn.set_context('talk')

plt.figure(figsize=(10,6))

pressure_plot2=np.logspace(-21,-9,40)
theta_hobson=np.exp(-b_hobson*(gas_constant*4.2*np.log(pressure_plot2/(p_0*1e-2)))**2)
theta_wilfert=np.exp(-d*(constants.k*4.2*np.log(pressure_plot2/(p_0*1e-2)))**2)

# plt.loglog(data['Coverage (molecules/cm^2).1']/(3e15), data['Pressure (mbar).1'], '.--' )
# plt.errorbar(data['Coverage (molecules/cm^2).1']/(3e15), data['Pressure (mbar).1'],xerr=data['Coverage (molecules/cm^2).1']/(3e15)*0.20, yerr= data['Pressure (mbar).1']*0.99,fmt='^-',color='brown', alpha=0.5)
plt.loglog(data['Coverage (molecules/cm^2).1']/(3e15), data['Pressure (mbar).1'],'^-',color='brown',label='Fleur\'s data 1')
# plt.errorbar(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure Barion gauge (mbar)'],xerr=data['Coverage (molecules/cm^2).2']/(3e15)*0.20, yerr= data['Pressure Barion gauge (mbar)']*0.99,fmt='o-',color='orange',alpha=0.5)
plt.loglog(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure Barion gauge (mbar)'],'o-',color='orange' ,label='Fleur\'s data Barion gauge' )
# plt.loglog(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure RGA (mbar)'], '.--', label='Fleur\'s data RGA')
# plt.errorbar(data_wallen['Relative surface coverage'][4:],data_wallen['Pressure [mbar]'][4:],xerr=data_wallen['Relative surface coverage'][4:]*0.20,yerr=data_wallen['Pressure [mbar]'][4:]*0.99, fmt='g-',alpha=0.5)
plt.loglog(data_wallen['Relative surface coverage'][4:],data_wallen['Pressure [mbar]'][4:], 'gD-', label='Wallen\'s data on Cu plated stainless steel')
plt.loglog(data_wallen['Relative surface coverage'][:4],data_wallen['Pressure [mbar]'][:4], 'g*', label='Wallen\'s data less than 10% over background pressure')

plt.loglog(theta,p_cold*1e-2, 'D',color='purple', label='Estimated limit point from BASE')
plt.fill_between(x=[4,theta],y1=[p_cold*1e-2,p_cold*1e-2],y2=[1e-21,1e-21],color='purple',alpha=0.5)

plt.loglog(theta_hobson,pressure_plot2, '--',color='royalblue', label='Prediction Hobson')
plt.loglog(theta_wilfert, pressure_plot2, '-.', color='lightskyblue', label='Prediction from fit of data from Chill, Wilfert and Bozyk')

plt.ylim(1e-21)
plt.xlim(2e-3,4)
plt.ylabel('Pressure in mbar')
plt.xlabel(r'Relative surface coverage')
plt.title('Hydrogen on copper at 4.2K')
plt.grid(axis='both', which='both')
plt.legend(loc=6, fontsize='small')
plt.tight_layout()


In [44]:
# %matplotlib qt
plt.figure()
# plt.rcParams['font.size'] = '25'

pressure_plot2=np.logspace(-19,-8,40)
theta_hobson=np.exp(-b_hobson*(gas_constant*4.2*np.log(pressure_plot2/(p_0*1e-2)))**2)
theta_wilfert=np.exp(-d*(constants.k*4.2*np.log(pressure_plot2/(p_0*1e-2)))**2)

# plt.loglog(theta_hobson,pressure_plot2, '-',color='royalblue', label='Prediction Hobson')
# plt.loglog(theta_wilfert, pressure_plot2, '-', color='lightskyblue', label='Prediction from fit of data from Chill, Wilfert and Bozyk')

plt.loglog(data['Coverage (molecules/cm^2).1']/(3e15), data['Pressure (mbar).1'], '.--', label='Fleur\'s data 1')
plt.loglog(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure Barion gauge (mbar)'], '.--', label='Fleur\'s data Barion')
plt.loglog(data['Coverage (molecules/cm^2).2']/(3e15), data['Pressure RGA (mbar)'], '.--', label='Fleur\'s data RGA')
plt.loglog(data_wallen['Relative surface coverage'][4:],data_wallen['Pressure [mbar]'][4:], 'r.-', label='Wallen\'s data on Cu plated stainless steel')
plt.loglog(data_wallen['Relative surface coverage'][:4],data_wallen['Pressure [mbar]'][:4], 'r*', label='Wallen\'s data less than 10% over background pressure')

plt.loglog(theta,p_cold*1e-2, 'D', label='Estimated data point from BASE')
plt.ylabel('Pressure in mbar')
plt.xlabel(r'Relative surface coverage')
plt.title('Hydrogen on copper at 4.2K')
plt.grid(axis='both', which='minor')
plt.legend();

## Huang's theory
I was not able to retrieve a "formula" of an adsorption isotherm from the papers...

## Assumptions
* Hydrogen fully pumped before pinch off, no sticking to the surfaces $\rightarrow$ Hydrogen only from outgassing between pinch off and cooling
* Outgassing negligible when system is cold

## Open questions
* Partial pressure of hydrogen before pinch off
* Hydrogen release during pinch off
* 

## Inverse function

In [ ]:
# saturation pressure in mbar
p_saturation={3: 8.5e-12*1.333, 4.2: p_0_mbar, 6: 9.3e-4*1.333, 10: 1.91*1.333, 15: 95*1.333, 20: 7.26e2*1.333}

In [ ]:
def p_sat(temp):
    try:
        return p_saturation[temp]
    except KeyError:
        return 'Temperature missing in dictionary'
def theta(temp, pressure):
    try:
        return np.exp(-d*(constants.k*temp*np.log(pressure/p_sat(temp)))**2)
    except:
        print(p_sat(temp))
def theta(temp, pressure=[]):
    try:
        return np.exp(-d*(constants.k*temp*np.log(pressure/p_sat(temp)))**2)
    except:
        print(p_sat(temp))
def pressure(temp, theta=[]):
    try:
        return p_sat(temp)*np.exp(-1./(np.sqrt(d)*constants.k*temp)*np.sqrt(-np.log(theta)))
    except:
        print(p_sat(temp))

In [ ]:
theta(4.2,1e-17)

In [ ]:
pressure(4.2,0.7856502482339683)